### Reporte imagenes SAR con empleando transformers

Se emplearon 2 arquitecturas para la restauracion de imagenes y reduccion de ruido implementando los siguientes papers:

- [Restormer: Efficient Transformer for High-Resolution Image Restoration](https://arxiv.org/pdf/2111.09881)
- [Uformer: A General U-Shaped Transformer for Image Restoration](https://openaccess.thecvf.com/content/CVPR2022/papers/Wang_Uformer_A_General_U-Shaped_Transformer_for_Image_Restoration_CVPR_2022_paper.pdf)

El reporte consta de la exploracion de la siguiente arquitectura:

- [Restormer](#Restormer)
    - [Entrenamiento](#r-entrenamiento)
    - [Cargar Modelo](#r-modelo)
    - [Inferecias](#r-inferencias)
    - [Metricas](#r-metricas)

- [Uformer](#Uformer)
    - [Entrenamiento](#u-entrenamiento)
    - [Inferecias](#u-inferencias)
    - [Metricas](#u-metricas)

- [Metricas comparativas](#metricas-comparativas)
- [Test](#test)

In [ ]:
from IPython.display import HTML
HTML("""<style>
.column {
  float: left;
  width: 50%;
}

/* Clear floats after the columns */
.row:after {
  content: "";
  display: table;
  clear: both;
}
</style>
<div class="row">
  <div class="column"><iframe src=../references/Wang_Uformer_A_General_U-Shaped_Transformer_for_Image_Restoration_CVPR_2022_paper.pdf width=700 height=700></iframe></div>
  <div class="column"><iframe src=../references/2111.09881v2.pdf width=700 height=700></div>
</div>""")

# Variable de la carpeta principal
A a hora de ejercutar este archivo, por favor reemplazar esta variable con la ruta de donde se descargo.

In [1]:
from pathlib import Path
root_folder = Path("/home/arthemis/Documents/pytorch_env/pytorch_env/transformers/")

# Restormer

![alt text](../../references/restormer.png)

### Entrenamiento <a id='r-entrenamiento'></a> 

Los parametro de configuracion estan ubicados en el siguiente archivo:
- [Configuracion](../data/config/config.yml)

In [ ]:

# Modify the file in /home/arthemis/Documents/pytorch_env/pytorch_env/transformers_ruben/src/data/config/config.yml
# Restormer
import os
wd = os.getcwd()
os.chdir("../../")
%run basicsr/train.py
os.chdir(wd)

## Cargar Modelo <a id='r-modelo'></a> 

A la hora de escoger un nuevo modelo entrenado con otra perdida,cambiar el valor de PSNR por CHARBONNIER o L1LOSS

```python
class ModelsName(str, Enum):
    """Clase Enum para escoger tipo de perdida para el modelo Restormer"""
    PSNR = "PSNR"
    CHARBONNIER = "CharbonierLoss"
    L1LOSS = "L1Loss_0_99"

# Ejemplo PSRN
model_name = ModelsName.PSNR.value
# Charbonnier
model_name = ModelsName.CHARBONNIER.value
# L1LOSS
model_name = ModelsName.L1LOSS.value


weights, parameters = get_weights_and_parameters(model_name)
```


**PSNR (Peak Signal-to-Noise Ratio)**

- El PSNR se calcula como el logaritmo base 10 de la relación entre la máxima potencia posible de la señal y la potencia del ruido de fondo. Matemáticamente, se expresa como:

$ PSNR = 10 \log_{10}( \frac{\text{MAX PIXEL VALUE}^2 }{MSE} ) $

Donde:
- MAX PIXEL VALUE es el valor máximo posible para un píxel (por ejemplo, 255 para imágenes de 8 bits)
- MSE es el error cuadrático medio (Mean Squared Error) entre las imágenes original y reconstruida

Ventajas:
- Fácil de interpretar y comprender.
- Ampliamente utilizada y aceptada en la comunidad de investigación de imágenes.

Desventajas:
- Sensible a cambios en la luminosidad global de la imagen.
- No tiene en cuenta la percepción visual humana.

**Charbonnier Loss**

La función de pérdida Charbonnier, también conocida como " pérdida robusta de L1", es una alternativa al PSNR que ofrece mayor robustez a valores atípicos (outliers) en los datos. En lugar de penalizar linealmente los errores, la pérdida Charbonnier aplica una penalización suave, lo que la hace menos sensible a picos de ruido.

$ CharbonnierLoss = \alpha \cdot \sqrt{(x^2 + \epsilon^2)} + (1 - \alpha) \cdot L1_{Loss}(x) $

Donde:

- x es la diferencia entre los valores de píxeles correspondientes en las imágenes original y reconstruida
- alpha es un factor de ponderación entre la penalización suave y la penalización L1 (típicamente entre 0 y 1)
- epsilon es un pequeño valor positivo que evita la división por cero

Ventajas:
- Robusta a valores atípicos en los datos.
- Puede producir imágenes con mayor nitidez y detalles.

Desventajas:
- Cálculo ligeramente más complejo que el PSNR.
- La elección del parámetro alpha puede afectar significativamente el resultado.


**L1 Loss (L1 Loss)**

La función de pérdida L1 (L1 Loss), también conocida como "pérdida absoluta", es una medida simple y directa de la diferencia entre los valores de píxeles correspondientes en las imágenes original y reconstruida. Se calcula como la suma de los valores absolutos de las diferencias de píxeles.

La función de pérdida L1 se define como:

$ L1_{Loss} = sum(abs(x)) $

Donde:

- x es la diferencia entre los valores de píxeles correspondientes en las imágenes original y reconstruida
- abs() es la función valor absoluto

Ventajas:
- Cálculo simple y eficiente.
- Robusta a valores atípicos en los datos.

Desventajas:
- No tiene en cuenta la percepción visual humana.
- Puede producir imágenes borrosas o con artefactos.

In [2]:
import torch
import torch.nn.functional as F
from runpy import run_path
import cv2
from tqdm import tqdm
import numpy as np
from pathlib import Path
import os
from enum import Enum

# Cambiar cantidad de epoca de los pesos
def get_weights_and_parameters(model: str, folder_weigths:  Path | str)-> dict[str, list[Path] | list[int]]:

    # Si se quiere correr otro modelo se requiere cambiar el path
    models = {"CharbonierLoss": {
        "weights_available" : (weights_available:=list(range(4000, 88000, 4000))),
        "path": [Path(f"{folder_weigths}/CharbonierLoss/net_g_{w}.pth") for w in weights_available]
    },
    "L1Loss_0_99":{
        "weights_available" : (weights_available:=list(range(4000, 84000, 4000))),
        "path": [Path(f"{folder_weigths}/L1Loss_0_99/net_g_{w}.pth") for w in weights_available]
    },
   "PSNR": {
        "weights_available" : (weights_available:=list(range(4000, 104000, 4000))),
        "path": [Path(f"{folder_weigths}/PSNR/net_g_{w}.pth") for w in weights_available]
    }    
    }
    
    return models[model]


class ModelsName(str, Enum):
    """Clase Enum para escoger tipo de perdida para el modelo Restormer"""
    PSNR = "PSNR"
    CHARBONNIER = "CharbonierLoss"
    L1LOSS = "L1Loss_0_99"


*Definicion de rutas de pesos y scripts*

- folder_weigths_root : Ruta donde se encuetran los pesos para el modelo de Restormer
- model_path_root : Ruta donde se encuentra el script de ejecucion para cargar la arquitectura del modelo

In [8]:
folder_weigths_root = root_folder/ "src/data/model"
parameters = {'inp_channels':1, 'out_channels':1, 'dim':48, 'num_blocks':[4,6,6,8], 'num_refinement_blocks':4, 'heads':[1,2,4,8], 'ffn_expansion_factor':2.66, 'bias':False, 'LayerNorm_type':'BiasFree', 'dual_pixel_task':False}
model_name = ModelsName.L1LOSS.value
weights = get_weights_and_parameters(model_name, folder_weigths_root)

#Cargar arquitectura del modelo
model_path_root = root_folder/ "basicsr"
load_arch = run_path(os.path.join( model_path_root, "models", "archs","restormer_arch.py"))
model = load_arch['Restormer'](**parameters)
model.cuda()

Restormer(
  (patch_embed): OverlapPatchEmbed(
    (proj): Conv2d(1, 48, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
  )
  (encoder_level1): Sequential(
    (0): TransformerBlock(
      (norm1): LayerNorm(
        (body): BiasFree_LayerNorm()
      )
      (attn): Attention(
        (qkv): Conv2d(48, 144, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (qkv_dwconv): Conv2d(144, 144, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=144, bias=False)
        (project_out): Conv2d(48, 48, kernel_size=(1, 1), stride=(1, 1), bias=False)
      )
      (norm2): LayerNorm(
        (body): BiasFree_LayerNorm()
      )
      (ffn): FeedForward(
        (project_in): Conv2d(48, 254, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (dwconv): Conv2d(254, 254, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=254, bias=False)
        (project_out): Conv2d(127, 48, kernel_size=(1, 1), stride=(1, 1), bias=False)
      )
    )
    (1): TransformerBlock(
 

### Inferecias  <a id='r-inferencias'></a> 




*Definicion de rutas*

- input_dir : Directorio donde se almacenas las imagenes Ground Truth para validacion 
- noisy_dir : Directorio donde se almacenan las imagenes con Ruido o en su defecto sobre las cuales de hacen las predicciones
- out_dir : Directorio donde se almacenan la salida del modelo, tanto las imagenes como las metricas


**Nota:** Verificar el formato de las imagenes antes de ejecutar las celda siguiente dado se trabajo en PNG

In [9]:
from numpy import float64
from collections import defaultdict
from skimage.metrics import structural_similarity as ssim
from skimage.metrics import peak_signal_noise_ratio as psnr
from skimage.metrics import mean_squared_error as mse
import json

input_dir = root_folder / "src/data/validation_data_report"
noisy_dir = root_folder /"Uformer/images"
out_dir = root_folder / f"src/data/model/{model_name}/images"
os.makedirs(out_dir, exist_ok=True)

#Listado de imagenes seleccionadas en base al paper 
# files = ['0_1024', '5120_19456','5120_19968','5120_20480']
# files = ['0_1024', '5120_19456', '5120_512', '5632_25088', '5632_18944', '5632_11776', '5120_20992', '5120_3072', '5120_1536', '5120_17408', '5120_24064', '5632_7680']
files = ["2560_22016_Tokyo", "2560_23552_Tokyo", "0_512", "0_1024", "0_1536"]

img_multiple_of = 8

metrics = defaultdict(list[dict[str, float | str | float64]])
with torch.no_grad():
    for weight, index in zip(weights["path"], weights["weights_available"]):
        checkpoint = torch.load(weight)
        model.load_state_dict(checkpoint["params"])
        model.eval()
        for filepath in tqdm(files):
            torch.cuda.ipc_collect()
            torch.cuda.empty_cache()
            # im_path: Path = noisy_dir / f"{filepath}.png"
            im_path: Path = noisy_dir / f"{filepath}.tiff"
            og = cv2.imread(im_path.as_posix(),-1)
            img = cv2.cvtColor(og, cv2.COLOR_BGR2GRAY)
            img_ = np.reshape(img, (512, 512, 1))

            input_ = torch.from_numpy(img_).float().permute(2, 0, 1).unsqueeze(0).cuda()
            h, w = input_.shape[2], input_.shape[3]
            H, W = (
                ((h + img_multiple_of) // img_multiple_of) * img_multiple_of,
                ((w + img_multiple_of) // img_multiple_of) * img_multiple_of,
            )
            padh = H - h if h % img_multiple_of != 0 else 0
            padw = W - w if w % img_multiple_of != 0 else 0
            input_ = F.pad(input_, (0, padw, 0, padh), "reflect")
            restored = model(input_)

            # Unpad the output
            restored = restored[:, :, :h, :w]
            restored = restored.permute(0, 2, 3, 1).cpu().detach().numpy()
            restored = np.squeeze(restored[0])

            max_retormer = np.max(restored)
            min_retormer = np.min(restored)     
            # val_im_path: Path = input_dir / f"{filepath}.tiff"
            # val_og = cv2.imread(val_im_path.as_posix(),-1)
            # val_img = cv2.cvtColor(val_og, cv2.COLOR_BGR2GRAY)
            
            # metrics[f"model_{index}"].append({
			# 	f"file_{filepath}": "",
			# 	"psnr": psnr(val_img.astype(np.float32), restored, data_range=max_retormer - min_retormer),
			# 	"ssim": ssim(val_img.astype(np.float32), restored, data_range=max_retormer - min_retormer),
			# 	"mse": mse(val_img.astype(np.float32), restored)
			# })
            filename = os.path.split(input_dir)[-1]
            cv2.imwrite(os.path.join(out_dir, f"{filepath}_{index}.png"), restored)
save_dir_metrics = out_dir / "".join((model_name,".json"))
# json.dump(metrics,save_dir_metrics.open("w"))

100%|██████████| 5/5 [00:02<00:00,  2.06it/s]


### Metricas  <a id='r-metricas'></a> 

In [6]:
def calculate_best_model(data):
	"""
	Analyzes a JSON structure containing model performance data and identifies
	the model with the best overall performance based on:
	    - Lowest PSNR (Peak Signal-to-Noise Ratio)
	    - Highest SSIM (Structural Similarity Index Measure)
	    - Lowest MSE (Mean Squared Error)

	Args:
	    data (dict): A dictionary representing the JSON data containing
	                 model performance information.

	Returns:
	    str: The name of the model with the best overall performance.
	"""

	best_model = None
	best_score = float("inf")  # Initialize with positive infinity

	for model_name, model_data in data.items():
		for entry in model_data:
			score = entry["psnr"] + (1 - entry["ssim"]) + entry["mse"]
			# Combine PSNR, SSIM (inverted), and MSE into a single score

			if score < best_score:
				best_model = model_name
				best_score = score

	return best_model

metrics = json.load(save_dir_metrics.open('r'))
print(calculate_best_model(metrics))

model_76000


# Uformer

![alt text](../../Uformer/fig/Uformer.png)

### Entrenamiento <a id='u-entrenamiento'></a> 

Los parametro de configuracion estan ubicados en el siguiente archivo:
- [Configuracion](../data/config/config.yml)

*Ruta para ejecucion*
- train_workers : "Ruta de las imagenes Ground Truth"
- val_dir: "Ruta de la imagenes Noisy",
- gpu: Numero de la GPU para ejecucion, por defecto empieza en la posicion cero

In [ ]:
import os
wd = os.getcwd()
os.chdir("../Uformer")
! poetry run train/train_denoise.py --train_dir /home/arthemis/Documents/pytorch_env/pytorch_env/transformers_ruben/src/data/transformed_data/GTruth/ --gpu 0 --val_dir /home/arthemis/Documents/pytorch_env/pytorch_env/transformers_ruben/src/data/transformed_data/val
os.chdir(wd)

### Inferecias  <a id='u-inferencias'></a> 

*Ruta para ejecucion*
- input_dir : "Ruta de las imagenes Noisy"
- result_dir: "Ruta donde se guardan las imagenes",
- weights: Ruta de los pesos del modelo entrenado en la celda anterior


In [14]:
import os
wd = os.getcwd()
os.chdir("../Uformer")
! poetry run python3 test/test_dnd.py --input_dir /home/arthemis/Documents/pytorch_env/pytorch_env/transformers/Uformer/images --result_dir ./results/denoising/DND/ --weights /home/arthemis/Documents/pytorch_env/pytorch_env/transformers/Uformer/logs/denoising/SIDD/Uformer_B_/models/model_best.pth
os.chdir(wd)

You choose Uformer_B...
/home/arthemis/.cache/pypoetry/virtualenvs/pytorch-env-VZ5w7g_q-py3.10/lib/python3.10/site-packages/torch/functional.py:507: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at ../aten/src/ATen/native/TensorShape.cpp:3549.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]
===>Testing using weights:  /home/arthemis/Documents/pytorch_env/pytorch_env/transformers/Uformer/logs/denoising/SIDD/Uformer_B_/models/model_best.pth
  0%|                                                     | 0/5 [00:00<?, ?it/s]0
0_1024
 20%|█████████                                    | 1/5 [00:00<00:02,  1.48it/s]1
0_1536
 40%|██████████████████                           | 2/5 [00:00<00:01,  2.68it/s]2
0_512
 60%|███████████████████████████                  | 3/5 [00:00<00:00,  3.62it/s]3
2560_22016_Tokyo
 80%|████████████████████████████████████         | 4/5 [00:01<00:00,  4.34it/s]4
2560_2

### Metricas <a id='u-metricas'></a>

In [ ]:
import os
from pathlib import Path
import cv2
import numpy as np
from skimage.metrics import structural_similarity as ssim
from skimage.metrics import peak_signal_noise_ratio as psnr
from skimage.metrics import mean_squared_error as mse
from collections import defaultdict
from numpy import float64
from tqdm import tqdm

def metrics(gt_dir: Path = root_folder / 'src/data/validation_data_report',
            input_dir: Path = root_folder / "Uformer/results/denoising/DND/png",
            files: list[str] = ['0_1024', '5120_19456','5120_19968','5120_20480']):

    metrics_uformer = defaultdict(list[dict[str, float | str | float64]])
    for filepath in tqdm(files):
        im_path = gt_dir / f"{filepath}.tiff"
        og = cv2.imread(im_path.as_posix(),-1)
        img = cv2.cvtColor(og, cv2.COLOR_BGR2GRAY)
        # img_ = np.reshape(img, (512, 512, 1))
        restored_img = input_dir / f"{filepath}.png"
        restored = cv2.imread(restored_img.as_posix(),-1)
        restored = cv2.cvtColor(restored, cv2.COLOR_BGR2GRAY)
        # restored = np.reshape(img, (512, 512, 1))
        max_retormer: float = np.max(restored)
        min_retormer: float = np.min(restored)
        metrics_uformer["model"].append({
            f"file_{filepath}": "",
            "psnr": psnr(img.astype(np.float32), restored, data_range=max_retormer - min_retormer),
            "ssim": ssim(img.astype(np.float32), restored, data_range=max_retormer - min_retormer),
            "mse": mse(img.astype(np.float32), restored)
        })
        


    with open("metrics_uformer.json","w") as json_file:
        json.dump(metrics_uformer,json_file)

